In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

%matplotlib inline

PATH = '../data/preprocess/'
PATH_new = '../data/'
file_name = 'third_data.xlsx'

In [2]:
# 진단일자는 전처리를 위해 넣어준다.
init_columns = ['연구등록번호','Diagnosis','진단일자','Gender','진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

feature_column = ['Diagnosis','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

In [3]:
# Read data
df = pd.read_excel(PATH + file_name)
df = df[init_columns]


In [4]:
# Total number
print(df.shape)

# Info
print(df.info())

# Count Diagnosis 0,1,2 
print(df['Diagnosis'].value_counts())

# Diagnosis 기준 count
print(df.groupby('Diagnosis').count())



(9148, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      9148 non-null   int64  
 1   Diagnosis                   9148 non-null   int64  
 2   진단일자                        9148 non-null   object 
 3   Gender                      9148 non-null   int64  
 4   진단시점나이                      9148 non-null   int64  
 5   CMV IgM[Serum]              844 non-null    float64
 6   CMV IgG[Serum]              817 non-null    float64
 7   HSV IgM[Serum]              872 non-null    float64
 8   HSV IgG[Serum]              649 non-null    float64
 9   VZV IgM[Serum]              757 non-null    float64
 10  VZV IgG[Serum]              757 non-null    float64
 11  WBC COUNT[Whole blood]      8852 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8804 non-null   float64
 13  Lymphocyte(%)[Whole bl

In [5]:
# '환자' number
print(df['연구등록번호'].nunique())

2914


## Multivariate Imputation

In [22]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [23]:
def impute_missing_values(df, max_iter = 100):
    # Iterative Imputer
    # lr = LinearRegression()
    # imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=max_iter, verbose=2, imputation_order='roman',random_state=0)
    imp = IterativeImputer(missing_values=np.nan, max_iter=max_iter, verbose=2, imputation_order='roman',random_state=0)
    df_return = imp.fit_transform(df)
    return df_return

In [24]:
MI_feature_column = ['연구등록번호','Diagnosis','CMV IgM[Serum]','CMV IgG[Serum]',
'HSV IgM[Serum]','HSV IgG[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]',
'Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]',
'Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

In [25]:
df_group = df[MI_feature_column].groupby('연구등록번호')
for name, group in df_group:
    # check the group with Nan
    list_nan = []
    for i in group.columns:
        nan_num = group[i].isnull().sum()
        if nan_num >= 0 and nan_num < len(group):
            list_nan.append(i)
    
    # # impute the Nan
    # if len(list_nan) > 0:
    #     print(group.shape)
    #     print(group)
    #     group_imp = impute_missing_values(group)
    #     print('-------------------')
    #     print(group_imp.shape)
    #     print(group_imp)
    #     df.loc[df['연구등록번호'] == name, list_nan] = group_imp
    #     print('-------------------')
    #     print(df.loc[df['연구등록번호'] == name, list_nan])
    # impute the Nan
    if len(list_nan) > 0:
        group_imp = impute_missing_values(group)
        df.loc[df['연구등록번호'] == name, list_nan] = group_imp
    
    

[IterativeImputer] Completing matrix with shape (2, 8)
[IterativeImputer] Ending imputation round 1/100, elapsed time 0.01
[IterativeImputer] Change: 0.0, scaled tolerance: 104.15 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (6, 16)
[IterativeImputer] Ending imputation round 1/100, elapsed time 0.04
[IterativeImputer] Change: 0.0, scaled tolerance: 261.562 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (1, 15)
[IterativeImputer] Ending imputation round 1/100, elapsed time 0.02
[IterativeImputer] Change: 0.0, scaled tolerance: 282.509 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (1, 10)
[IterativeImputer] Ending imputation round 1/100, elapsed time 0.01
[IterativeImputer] Change: 0.0, scaled tolerance: 349.44 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (1, 8)
[Ite

In [26]:
print(df.info())

# save data
df.to_excel(PATH + 'MI1.xlsx')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   연구등록번호                      9148 non-null   float64
 1   Diagnosis                   9148 non-null   float64
 2   진단일자                        9148 non-null   object 
 3   Gender                      9148 non-null   int64  
 4   진단시점나이                      9148 non-null   int64  
 5   CMV IgM[Serum]              1926 non-null   float64
 6   CMV IgG[Serum]              1880 non-null   float64
 7   HSV IgM[Serum]              2128 non-null   float64
 8   HSV IgG[Serum]              1584 non-null   float64
 9   VZV IgM[Serum]              1787 non-null   float64
 10  VZV IgG[Serum]              1878 non-null   float64
 11  WBC COUNT[Whole blood]      9030 non-null   float64
 12  Lymphocyte(#)[Whole blood]  8992 non-null   float64
 13  Lymphocyte(%)[Whole blood]  8992 

## 환자 기준으로 바꿔서 진행

In [34]:
# '연구등록번호' 기준 group
df_group = df.groupby('연구등록번호')
df_group_mean = df_group.mean()
print(df_group_mean.info())


<class 'pandas.core.frame.DataFrame'>
Float64Index: 2914 entries, 104150.0 to 10719371.0
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Diagnosis                   2914 non-null   float64
 1   Gender                      2914 non-null   float64
 2   진단시점나이                      2914 non-null   float64
 3   CMV IgM[Serum]              830 non-null    float64
 4   CMV IgG[Serum]              804 non-null    float64
 5   HSV IgM[Serum]              853 non-null    float64
 6   HSV IgG[Serum]              632 non-null    float64
 7   VZV IgM[Serum]              742 non-null    float64
 8   VZV IgG[Serum]              739 non-null    float64
 9   WBC COUNT[Whole blood]      2800 non-null   float64
 10  Lymphocyte(#)[Whole blood]  2771 non-null   float64
 11  Lymphocyte(%)[Whole blood]  2771 non-null   float64
 12  Monocyte(#)[Whole blood]    2770 non-null   float64
 13  Monocyte(%)[Whole 

In [35]:
# 'CMV IgM[Serum]' , 'Lymphocyte(#)[Whole blood]' 가 다 결측치인 case drop
df_group_mean = df_group_mean.dropna(subset=['CMV IgM[Serum]','Lymphocyte(#)[Whole blood]'],how='all')
print(df_group_mean.info())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2858 entries, 104150.0 to 10719371.0
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Diagnosis                   2858 non-null   float64
 1   Gender                      2858 non-null   float64
 2   진단시점나이                      2858 non-null   float64
 3   CMV IgM[Serum]              830 non-null    float64
 4   CMV IgG[Serum]              803 non-null    float64
 5   HSV IgM[Serum]              849 non-null    float64
 6   HSV IgG[Serum]              626 non-null    float64
 7   VZV IgM[Serum]              736 non-null    float64
 8   VZV IgG[Serum]              732 non-null    float64
 9   WBC COUNT[Whole blood]      2783 non-null   float64
 10  Lymphocyte(#)[Whole blood]  2771 non-null   float64
 11  Lymphocyte(%)[Whole blood]  2771 non-null   float64
 12  Monocyte(#)[Whole blood]    2770 non-null   float64
 13  Monocyte(%)[Whole 

In [36]:
#save data
df_group_mean.to_excel(PATH + 'mean_patient.xlsx')

## Diagnosis 기준으로 multivariate imputation 진행

In [37]:
df_new_group = df_group_mean.groupby('Diagnosis')
for name, group in df_new_group:
    list_nan = []
    for i in group.columns:
        nan_num = group[i].isnull().sum()
        if nan_num >= 0 and nan_num < len(group):
            list_nan.append(i)
    
    if len(list_nan) > 0:
        group_imp = impute_missing_values(group, max_iter = 1000)
        df_group_mean.loc[df_group_mean['Diagnosis'] == name, list_nan] = group_imp

[IterativeImputer] Completing matrix with shape (2662, 18)
[IterativeImputer] Ending imputation round 1/1000, elapsed time 0.11
[IterativeImputer] Change: 475.60102770438897, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 2/1000, elapsed time 0.19
[IterativeImputer] Change: 575.1004604745841, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 3/1000, elapsed time 0.28
[IterativeImputer] Change: 1632.8679495868068, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 4/1000, elapsed time 0.36
[IterativeImputer] Change: 969.0832312995152, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 5/1000, elapsed time 0.44
[IterativeImputer] Change: 467.4624426087347, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 6/1000, elapsed time 0.51
[IterativeImputer] Change: 324.77282755183415, scaled tolerance: 8.5 
[IterativeImputer] Ending imputation round 7/1000, elapsed time 0.59
[IterativeImputer] Change: 292.1977333511

/Users/kwontaeyoon/opt/anaconda3/envs/DL/lib/python3.7/site-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


In [38]:
print(df_group_mean.info())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2858 entries, 104150.0 to 10719371.0
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Diagnosis                   2858 non-null   float64
 1   Gender                      2858 non-null   float64
 2   진단시점나이                      2858 non-null   float64
 3   CMV IgM[Serum]              2858 non-null   float64
 4   CMV IgG[Serum]              2858 non-null   float64
 5   HSV IgM[Serum]              2858 non-null   float64
 6   HSV IgG[Serum]              2858 non-null   float64
 7   VZV IgM[Serum]              2858 non-null   float64
 8   VZV IgG[Serum]              2858 non-null   float64
 9   WBC COUNT[Whole blood]      2858 non-null   float64
 10  Lymphocyte(#)[Whole blood]  2858 non-null   float64
 11  Lymphocyte(%)[Whole blood]  2858 non-null   float64
 12  Monocyte(#)[Whole blood]    2858 non-null   float64
 13  Monocyte(%)[Whole 

In [39]:
# save data
df_group_mean.to_excel(PATH + 'fourth_data.xlsx')

## 데이터 표준화

In [40]:
df_final = df_group_mean
# final data mean, median, std, min, max 저장
df_final_mean = df_final.mean()
df_final_median = df_final.median()
df_final_std = df_final.std()
df_final_min = df_final.min()
df_final_max = df_final.max()

# Write all into text file
detail_string = '''
--------------------Mean--------------------
{df_final_mean}
--------------------Median--------------------
{df_final_median}
--------------------Std--------------------
{df_final_std}
--------------------Min--------------------
{df_final_min}
--------------------Max--------------------
{df_final_max}
'''
write_file = open(PATH + 'final_data_detail.txt', 'w')
write_file.write(detail_string.format(df_final_mean=df_final_mean,
                                      df_final_median=df_final_median,
                                      df_final_std=df_final_std,
                                      df_final_min=df_final_min,
                                      df_final_max=df_final_max))

4026

In [41]:
normalization_feature = ['진단시점나이','CMV IgM[Serum]','CMV IgG[Serum]','HSV IgM[Serum]','VZV IgM[Serum]','VZV IgG[Serum]','WBC COUNT[Whole blood]','Lymphocyte(#)[Whole blood]','Lymphocyte(%)[Whole blood]','Monocyte(#)[Whole blood]','Monocyte(%)[Whole blood]','Neutrophil(#)[Whole blood]','Neutrophil(%)[Whole blood]','ESR[Whole blood]','CRP[Serum]']

# normalize features
df_normalize = df_final.copy()

df_normalize[normalization_feature] = df_normalize[normalization_feature].apply(lambda x: (x - x.mean()) / (x.std()))

In [42]:
df_normalize.head()

,Diagnosis,Gender,진단시점나이,CMV IgM[Serum],CMV IgG[Serum],HSV IgM[Serum],HSV IgG[Serum],VZV IgM[Serum],VZV IgG[Serum],WBC COUNT[Whole blood],Lymphocyte(#)[Whole blood],Lymphocyte(%)[Whole blood],Monocyte(#)[Whole blood],Monocyte(%)[Whole blood],Neutrophil(#)[Whole blood],Neutrophil(%)[Whole blood],ESR[Whole blood],CRP[Serum]
연구등록번호,,,,,,,,,,,,,,,,,,
104150.0,0.0,1.0,0.356129,-0.046668,-0.281828,-0.233945,58.835519,-0.096514,-0.145682,-0.190297,-0.459072,-0.313814,-0.220267,-0.146711,0.048839,0.156132,-0.139866,-0.061923
261562.0,1.0,0.0,1.327695,1.293737,-1.319948,-0.115558,23.400000,0.217760,-1.447486,1.609480,-0.560775,-1.583382,0.403957,-0.520290,2.627235,1.034753,-0.488559,-0.266494
282509.0,0.0,1.0,0.813337,0.489928,-1.192528,1.995004,35.084002,1.053658,-1.444090,-0.623532,0.181656,0.915729,-0.131092,0.346412,-0.858840,-1.459825,-1.121864,-0.269208
349440.0,0.0,1.0,0.184677,-0.100691,-0.299142,-0.135850,54.145101,-0.095873,-0.226626,-0.649877,-0.330926,0.243323,-0.238102,0.196980,-0.703717,-0.480169,-0.805212,-0.271923
386599.0,0.0,1.0,-0.729738,0.024528,-0.427286,-0.149510,38.728101,0.152072,-0.077436,-0.717204,-0.318722,0.406622,-0.666142,-0.370858,-0.686947,-0.507991,-0.606560,-0.405337


In [43]:
df_normalize.to_excel(PATH + 'final_data_normalize.xlsx')

In [44]:
print(df_final.describe())

         Diagnosis       Gender       진단시점나이  CMV IgM[Serum]  CMV IgG[Serum]  \
count  2858.000000  2858.000000  2858.000000     2858.000000     2858.000000   
mean      0.117565     0.509447    46.768616        0.044487       46.756574   
std       0.449204     0.499998    17.497530        0.119431       27.468177   
min       0.000000     0.000000     0.000000       -1.576156        0.000000   
25%       0.000000     0.000000    34.000000        0.000000       38.641304   
50%       0.000000     1.000000    48.000000        0.027527       45.260617   
75%       0.000000     1.000000    60.000000        0.060360       52.514348   
max       2.000000     1.000000    90.454545        1.730000      590.368188   

       HSV IgM[Serum]  HSV IgG[Serum]  VZV IgM[Serum]  VZV IgG[Serum]  \
count     2858.000000     2858.000000     2858.000000     2858.000000   
mean         0.227376       51.875182        0.147898      858.718283   
std          0.236904       30.164869        0.239264      5